## Access the database

In [1]:
# define parameters
db_name = 'dbsprojekt' # db should already be created locally
user = 'postgres'
password = 'password5432'

import_done = True # nach dem ersten Ausführen auf True setzen

In [2]:
import pandas as pd
# import numpy as np
import psycopg2

In [3]:
# connect to database
try:
    print('Connecting to the PostgreSQL database...')
    conn = psycopg2.connect(host = 'localhost', database = db_name, user = user, password = password)
    
    cur = conn.cursor()
    
    print('connected')
    
except (Exception, psycopg2.DatabaseError) as error:
    print(error)

Connecting to the PostgreSQL database...
connected


## Read in data files and create tables

In [6]:
# read files
f1 = open('literature_dataset.csv')
f2 = open('journal_dataset.csv')

# list of all attributes in each table
attr_list_lit = f1.readline().replace('\n', '').split(sep = ',')
attr_list_jif = f2.readline().replace('\n', '').split(sep = ',')

print('Attributes of the literature table: ' + str(attr_list_lit))
print('Attributes of the journals table: ' + str(attr_list_jif))

if not import_done:
    try:
        # create table for literature dataset
        # get list of attributes
        attr_list = ''
        for i in attr_list_lit:
            attr_list += '{} varchar, '.format(i)
        attr_list = attr_list[:-2] # entfernt ', ' am Ende der Liste

        # create literature table & import data
        sql_create = 'CREATE TABLE literature ( {} );'.format(attr_list)
        print(sql_create)
        cur.execute(sql_create)
        cur.copy_expert("""COPY literature FROM STDIN WITH CSV HEADER DELIMITER AS ','""", f1)
        
        # create table for journal dataset
        # get list of attributes        
        attr_list = ''
        for i in attr_list_jif:
            attr_list += '{} varchar, '.format(i)
        attr_list = attr_list[:-2]
        
        # create impact factors table & import data
        sql_create = 'CREATE TABLE journals ( {} );'.format(attr_list)
        print(sql_create)
        cur.execute(sql_create)
        cur.copy_expert("""COPY journals FROM STDIN WITH CSV HEADER DELIMITER AS ','""", f2)
    
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)

Attributes of the literature table: ['Titel', 'Autor', 'Jahr', 'Journal', 'Typ', 'DOI']
Attributes of the journals table: ['Title', 'Rank', 'Impact_Factor', 'IF_5_Yr', 'Half_Life', 'Percentage_Citable', 'Avg_IF_Percentile']


## Visualize data

In [10]:
# show joined table
sql_showtables = """
    SELECT *
    FROM literature L, journals J
    WHERE L.Journal = J.Title;
    """

In [11]:
try:
    cur.execute(sql_showtables)
    joined = cur.fetchall()
    joined = pd.DataFrame(data = joined, columns = attr_list_lit + attr_list_jif)

except (Exception, psycopg2.DatabaseError) as error:
    print(error)

In [12]:
joined.head(5)

,Titel,Autor,Jahr,Journal,Typ,DOI,Title,Rank,Impact_Factor,IF_5_Yr,Half_Life,Percentage_Citable,Avg_IF_Percentile
0,implementation of local area vr environment us...,"kim, soo-kyun; lee, chang-hee; kim, sun-jeong;...",2020,intelligent automation and soft computing,None,10.31209/2019.100000131,intelligent automation and soft computing,8692,1.276,1.177,4.0,100.0,23.999
1,distinction between real faces and photos by a...,"lee, byong kwon; lee, yang sun",2020,intelligent automation and soft computing,None,10.31209/2019.100000134,intelligent automation and soft computing,8692,1.276,1.177,4.0,100.0,23.999
2,cognitive load and performance in immersive vi...,"frederiksen, joakim grant; sorensen, stine may...",2020,surgical endoscopy and other interventional te...,None,10.1007/s00464-019-06887-8,surgical endoscopy and other interventional te...,2996,3.149,3.239,6.5,89.56,79.286
3,tracking attacks on virtual reality systems,"rafique, muhammad usman; cheung, sen-ching s.",2020,ieee consumer electronics magazine,None,10.1109/MCE.2019.2953741,ieee consumer electronics magazine,1870,4.016,3.148,2.6,100.0,79.616
4,advantages and limits of virtual reality in le...,"checa, david; bustillo, andres",2020,virtual reality,None,10.1007/s10055-019-00389-7,virtual reality,2304,3.634,2.941,6.4,91.67,75.081


In [14]:
print('Total rows after joining: ' + str(len(joined)))

Total rows after joining: 862


In [23]:
# can use after getting an error
# conn.rollback()

## Define queries

### Sorting
#### Sort by rank

In [43]:
order = 'ASC'
# order = 'DESC'

sql_sortrank = """
    SELECT L.Titel, Autor, J.Title, CAST (Rank AS INT), Impact_Factor, IF_5_Yr
    FROM literature AS L, journals AS J
    WHERE L.Journal = J.Title
    ORDER BY Rank {};
    """.format(order)

print(sql_sortrank)


    SELECT L.Titel, Autor, J.Title, CAST (Rank AS INT), Impact_Factor, IF_5_Yr
    FROM literature AS L, journals AS J
    WHERE L.Journal = J.Title
    ORDER BY Rank ASC;
    


#### Sort by year

In [48]:
# order = 'ASC'
order = 'DESC'

sql_sortyear = """
    SELECT L.Titel, Autor, CAST (Jahr AS INT), J.Title, Rank, IF_5_Yr
    FROM literature AS L, journals AS J
    WHERE L.Journal = J.Title
    ORDER BY Jahr {};
    """.format(order)

print(sql_sortyear)


    SELECT L.Titel, Autor, CAST (Jahr AS INT), J.Title, Rank, IF_5_Yr
    FROM literature AS L, journals AS J
    WHERE L.Journal = J.Title
    ORDER BY Jahr DESC;
    


### Filtering
#### Papers published after/during/before a certain year

In [60]:
filter_op = '='
# filter_op = '<='
# filter_op = '>='
filter_year = '2018'

sql_filteryear = """
    SELECT L.Titel, Autor, Jahr, J.Title, Rank, IF_5_Yr
    FROM literature AS L, journals AS J
    WHERE L.Journal = J.Title
    AND CAST (Jahr AS INT) {} {};
    """.format(filter_op, filter_year)

print(sql_filteryear)


    SELECT L.Titel, Autor, Jahr, J.Title, Rank, IF_5_Yr
    FROM literature AS L, journals AS J
    WHERE L.Journal = J.Title
    AND CAST (Jahr AS INT) = 2018;
    


#### Papers written by a certain author (filter using last names)

In [68]:
filter_author = 'steinicke'

sql_filterauthor = """
    SELECT L.Titel, Autor, J.Title, Rank, Impact_Factor, IF_5_Yr
    FROM literature AS L, journals AS J
    WHERE L.Journal = J.Title
    AND Autor LIKE '%{}%';
    """.format(filter_author)

print(sql_filterauthor)


    SELECT L.Titel, Autor, J.Title, Rank, Impact_Factor, IF_5_Yr
    FROM literature AS L, journals AS J
    WHERE L.Journal = J.Title
    AND Autor LIKE '%steinicke%'
    


#### Papers published in a journal within a certain ranking

In [72]:
filter_rank = '500'

sql_filterrank = """
    SELECT L.Titel, Autor, Jahr, J.Title, Rank, IF_5_Yr
    FROM literature AS L, journals AS J
    WHERE L.Journal = J.Title
    AND CAST (Rank AS INT) <= {};
    """.format(filter_rank)

print(sql_filterrank)


    SELECT L.Titel, Autor, Jahr, J.Title, Rank, IF_5_Yr
    FROM literature AS L, journals AS J
    WHERE L.Journal = J.Title
    AND CAST (Rank AS INT) <= 500;
    


## Execute query

In [73]:
# enter query to execute
sql_query = sql_filterrank

In [74]:
try:
    df = pd.read_sql_query(sql_query, conn)

except (Exception, psycopg2.DatabaseError) as error:
    print(error)

## Display results

In [75]:
# display part of the results
numrows = 10
df.head(numrows)

# display all
# df

,titel,autor,jahr,title,rank,if_5_yr
0,alohomora: motion-based hotword detection in h...,"gu, jiaxi; yu, zhiwen; shen, kele",2020,ieee internet of things journal,313,11.705
1,skin-integrated wireless haptic interfaces for...,"yu, xinge; xie, zhaoqian; yu, yang; lee, jungy...",2019,nature,15,46.486
2,a modality-independent network underlies the r...,"huffman, derek j.; ekstrom, arne d.",2019,neuron,170,15.998
3,practical chromatic aberration correction in v...,"zhan, tao; zou, junyu; xiong, jianghao; liu, x...",2020,advanced optical materials,435,8.224
4,a survey on 360 degrees video streaming: acqui...,"fan, ching-ling; lo, wen-chih; pai, yu-tung; h...",2019,acm computing surveys,463,9.456
5,scalable 360 degrees video stream delivery: ch...,"zink, michael; sitaraman, ramesh; nahrstedt, k...",2019,proceedings of the ieee,303,11.753
6,mixed reality in visceral surgery development ...,"sauer, igor m.; queisner, moritz; tang, peter;...",2017,annals of surgery,309,9.306
7,virtual reality improves embodiment and neurop...,"pozeg, polona; palluel, estelle; ronchi, rober...",2017,neurology,393,8.899
8,automated design of freeform imaging systems,"yang, tong; jin, guo-fan; zhu, jun",2017,light-science & applications,181,15.005
9,enabling focus cues in head-mounted displays,"hua, hong",2017,proceedings of the ieee,303,11.753


## Close the connection

In [76]:
try:
    conn.commit()

except (Exception, psycopg2.DatabaseError) as error:
    print(error)

In [77]:
cur.close()
print('connection closed')

connection closed
